In [1]:
from keras.datasets import cifar10
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
from torch.nn import Sequential, Conv2d, MaxPool2d, ReLU, BatchNorm2d, Linear
import torchvision
from unicodedata import normalize
from torch.utils.data import Dataset
import glob
import IPython

In [3]:
NUMBER_EPOCHS = 40
HANGEUL_EPOCHS = 20

In [5]:
dataset = os.listdir('/content/drive/My Drive/학습/딥러닝스터디_DASH/PerspectiveImages/')
len(dataset)

981

In [4]:
datapath = '/content/drive/My Drive/학습/딥러닝스터디_DASH/PerspectiveImages/'
for i in os.listdir(datapath):
  if not i[0].isdigit():
    os.remove(os.path.join(datapath, i))

In [6]:
def k_number(filepath):
  ff = np.fromfile(filepath, dtype=np.uint8)
  gray_img = cv2.imdecode(ff, cv2.IMREAD_GRAYSCALE)
  (thresh, im_bw) = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  return im_bw

In [7]:
def version_classifier(image):
  if image[10,10] == 0 :
    return 0
  else:
    return 1

In [8]:
class conv_block(torch.nn.Module):
    def __init__(self, in_channels,out_channels, kernel_size = 3,stride = 1, padding = 1, pool_width = 2):
        super(conv_block, self).__init__()
        model = []
        model.append(Conv2d(in_channels, out_channels, kernel_size = kernel_size, stride = stride, padding = padding))
        model.append(BatchNorm2d(out_channels))
        model.append(ReLU())
        model.append(MaxPool2d((pool_width, pool_width)))
        self.model = Sequential(*model)
        
    def forward(self, input_):
        return self.model(input_)
    
    
class number_classifier(torch.nn.Module):
    def __init__(self):
        super(number_classifier, self).__init__()
        self.block1 = conv_block(1, 16)
        self.block2 = conv_block(16, 32)
        self.block3 = conv_block(32, 64)
        self.block4 = conv_block(64,128)
        self.block5 = conv_block(128,256)
        self.fc = Linear(128*2*2, 10)
        
    def features(self, input_):
        x = self.block1(input_)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        return x
    
    def logits(self, features):
        return self.fc(features)
    
    def forward(self, input_):
        x = self.features(input_)
        x = x.reshape(x.shape[0], -1)
        return self.logits(x)
    
class hangeul_classifier(torch.nn.Module):
    def __init__(self):
        super(hangeul_classifier, self).__init__()
        self.block1 = conv_block(1, 16)
        self.block2 = conv_block(16,32)
        self.block3 = conv_block(32, 64)
        self.block4 = conv_block(64,128)
        self.fc = Linear(128*2*2,40)
        
    def features(self, input_):
        x= self.block1(input_)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        return x
    
    def logits(self, features):
        return self.fc(features)
    
    def forward(self, input_):
        x = self.features(input_)
        x = x.reshape(x.shape[0], -1)
        return self.logits(x)

In [9]:
transform = torchvision.transforms.Compose([torchvision.transforms.Resize(32),torchvision.transforms.ToTensor()])
train_data = torchvision.datasets.MNIST(root='/content/drive/My Drive/학습/딥러닝스터디_DASH/', download=True, train=True, transform=transform)
test_data = torchvision.datasets.MNIST(root='/content/drive/My Drive/학습/딥러닝스터디_DASH/', download=True, train=False, transform=transform)

In [10]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=512, shuffle=False)

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
number_model = number_classifier()
number_model.to(device)
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(number_model.parameters())

In [12]:
for i in range(NUMBER_EPOCHS):
    for image, label in train_loader:
        X_train = image.to(device)
        y_train = label.to(device)
        optimizer.zero_grad()
        y_pred = number_model(X_train)
        loss = loss_func(y_pred, y_train)
        loss.backward()
        optimizer.step()
    print("epoch : {}".format(i))

epoch : 0
epoch : 1
epoch : 2
epoch : 3
epoch : 4
epoch : 5
epoch : 6
epoch : 7
epoch : 8
epoch : 9
epoch : 10
epoch : 11
epoch : 12
epoch : 13
epoch : 14
epoch : 15
epoch : 16
epoch : 17
epoch : 18
epoch : 19
epoch : 20
epoch : 21
epoch : 22
epoch : 23
epoch : 24
epoch : 25
epoch : 26
epoch : 27
epoch : 28
epoch : 29
epoch : 30
epoch : 31
epoch : 32
epoch : 33
epoch : 34
epoch : 35
epoch : 36
epoch : 37
epoch : 38
epoch : 39


In [ ]:
number_model.eval()
with torch.no_grad():
  total = 0
  correct = 0
  for image, label in test_loader:
    X_test = image.to(device)
    y_test = label.to(device)

    output = number_model(X_test)
    _, output_idx = torch.max(output, 1)
    total += output.shape[0]
    correct += (output_idx == y_test).sum().float()
  print('accuracy : {}'.format(100*correct/total))

In [ ]:
image = cv2.imread(dataset[10])
#custom_config = r'--oem 3 --psm 6'
#pytesseract.image_to_string(image, config=custom_config)
print(image)

None


In [ ]:

image = Image.open('dataset[10]')
print(image)
custom_config = r'--oem 3 --psm 6'
pytesseract.image_to_string(image, config=custom_config)

FileNotFoundError: ignored

In [ ]:
#img = cv2.imread('image.jpg')

# Adding custom options
custom_config = r'--oem 3 --psm 6'
pytesseract.image_to_string(dataset[10], config=custom_config)

TesseractError: ignored

In [ ]:
img = cv2.imread(image,cv2.IMREAD_COLOR)
copy_img = img.copy()
img2 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

SystemError: ignored

In [ ]:
import cv2
import numpy as np

img = cv2.imread('/content/drive/My Drive/학습/딥러닝스터디_DASH/PerspectiveImages/01두6167.jpg')

# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

In [ ]:
gray = get_grayscale(image)
thresh = thresholding(gray)
opening = opening(gray)
canny = canny(gray)

TypeError: ignored

In [ ]:
from PIL import Image
import pytesseract
path = '/content/drive/My Drive/학습/딥러닝스터디_DASH/PerspectiveImages/58라8225.jpg'
img = Image.open(path)
img = img.convert('RGBA')
pix = img.load()
for y in range(img.size[1]):
    for x in range(img.size[0]):
        if pix[x, y][0] < 102 or pix[x, y][1] < 102 or pix[x, y][2] < 102:
            pix[x, y] = (0, 0, 0, 255)
        else:
            pix[x, y] = (255, 255, 255, 255)
img.save('temp.jpg')
text = pytesseract.image_to_string(Image.open('temp.jpg'))
# os.remove('temp.jpg')
print(text)

OSError: ignored

In [ ]:
tesseract '49가9267.jpg' stdout -l Hangul

SyntaxError: ignored

In [ ]:
!which tesseract

/usr/bin/tesseract


In [ ]:
pytesseract.pytesseract.tesseract_cmd = (
    r'/usr/bin/tesseract'
)

In [ ]:
image_ex = '/content/drive/My Drive/학습/딥러닝스터디_DASH/PerspectiveImages/86루6729.jpg'

extracted = pytesseract.image_to_string(Image.open(image_ex))

print(extracted)

TesseractNotFoundError: ignored

In [ ]:
img = cv2.imread('/Users/Sunny/python programming/PerspectiveImages/86루6729.jpg')

In [ ]:
 pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/pytesseract'